## Run setup

##### Ranges & challenges setup

In [10]:
import os
import random
import itertools

lvl_range           = [25, 60]
stat_range          = [20, 40]

n_lvl_challenge     = random.randint(0, 1)
n_stat_challenge    = random.randint(0, 1)
n_coventant_task    = random.randint(0, 2)
n_npc_challenges    = random.randint(0, 2)
n_boss_weapons      = random.randint(0, 2)

challenge_bosses    = random.randint(1, 3)     #### boss with a unqiue challenge
standard_bosses     = random.randint(1, 3)     #### boss with no challenge

boardSize           = 5     #### 5x5
entries             = pow(boardSize, 2)

n_current_challenges = n_stat_challenge + n_lvl_challenge + n_coventant_task + n_npc_challenges + n_boss_weapons + challenge_bosses + standard_bosses

recover_tasks       = int(entries - n_current_challenges)   #### Adaptive number of tasks

In [11]:
cPath = f"{os.getcwd()}" #### script's path

if "home" in(cPath):
    cPath = f"{os.getcwd()}/Challenges"
    lauching_from_gDrive    = "no"
    output_file = f"{cPath}/final_challenge_board.txt"

elif "content" in cPath:
    lauching_from_gDrive    = "yes"

    from google.colab import drive
    drive.mount('/content/drive')

    #### Main user path
    base_folder = "/content/drive/MyDrive/Games/DarkSouls/DarkSouls_Minigames"
    cPath = f"{base_folder}/Challenges"

    print(f"\n📁 Challenge path set to:\n{cPath}")


#### File reading & array definitions

In [12]:
boss_challenge_path = f"{cPath}/boss_challenge.txt"
boss_weapons_path   = f"{cPath}/boss_weapons.txt"
covenants_path      = f"{cPath}/covenants.txt"
npcs_path           = f"{cPath}/npcs.txt"
recover_path        = f"{cPath}/recover.txt"
weapon_type_path    = f"{cPath}/weapon_type.txt"
stats_path          = f"{cPath}/stats.txt"


startingClasses = ["Warrior","Knight","Wanderer","Thief","Bandit","Hunter","Sorcerer","Pyromancer","Cleric", "Deprived"]

startingGifts   = ["None","Divine Blessing","Black Firebomb","Twin Humanity","Binoculars","Pendant","Master Key","Tiny Being's Ring", "Old Witch's Ring"]

allBosses       = ["Moonlight Butterfly", "Pinwheel", "Taurus Demon", "Quelaag", "Bell Gargoyles", "Iron Golem", "Ceaseless Discharge", "Ornstein and Smough", "Four Kings", "Seath", "Bed of Chaos", "Nito", "Kalameet", "Artorias", "Manus", "Gwyn"]

npcOptions      = ["Kill", "Talk to"]

covenOptions    = ["Join" , "Betray"]


final_challenges = []


def load_list(path):
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]

boss_challenges = load_list(boss_challenge_path)
weapon_types    = load_list(weapon_type_path)
stats_list      = load_list(stats_path)
npc_list        = load_list(npcs_path)
covenants_list  = load_list(covenants_path)
recover_list    = load_list(recover_path)
boss_weapons    = load_list(boss_weapons_path)


#### Randomizations & defs

In [13]:
for _ in range(5):
    random.shuffle(boss_challenges)
    random.shuffle(weapon_types)
    random.shuffle(stats_list)
    random.shuffle(npc_list)
    random.shuffle(covenants_list)
    random.shuffle(recover_list)
    random.shuffle(boss_weapons)

    random.shuffle(startingClasses)
    random.shuffle(startingGifts)
    random.shuffle(allBosses)
    random.shuffle(npcOptions)
    random.shuffle(covenOptions)

def recover_task(recover_list):
    return random.choice(recover_list)

def infinite_random(items): #### used to randomize the choiche of challenges
    while True:
        yield random.choice(items)

def stats_check(lvl_cap , stat):
    if n_stat_challenge + n_lvl_challenge > 0:

        while lvl_cap < stat:
            lvl_cap += 1
            stat -= 1
            if lvl_cap >= stat:
                break


## Generating Challenges

In [14]:
lvl_cap = 0
stat    = 1

for _ in range(n_stat_challenge):
    stat            = random.randint(stat_range[0], stat_range[1])
    statString      = f"{random.choice(stats_list)} → {stat}"
    final_challenges.append(statString)

for _ in range(n_lvl_challenge):
    lvl_cap         = random.randint(lvl_range[0], lvl_range[1])
    lvl_capString   = f"Level Cap {lvl_cap}"
    final_challenges.append(lvl_capString)


stats_check(lvl_cap, stat)


for _ in range(n_boss_weapons):
    weapon      = random.choice(boss_weapons)
    boss_weapon = f"Forge {weapon}"
    boss_weapons.remove(weapon)
    final_challenges.append(boss_weapon)
    random.shuffle(boss_weapons)


for _ in range(n_coventant_task):
    covenant    = random.choice(covenants_list)
    action      = random.choice(covenOptions)
    covenants_list.remove(covenant)
    final_challenges.append(f"{action} {covenant}")
    random.shuffle(covenants_list)


for _ in range(n_npc_challenges):
    action      = random.choice(npcOptions)
    npc_pg      = random.choice(npc_list)
    npc_list.remove(npc_pg)
    final_challenges.append(f"{action} {npc_pg}")
    random.shuffle(npc_list)


for _ in range(recover_tasks):
    task        = random.choice(recover_list)
    recover_list.remove(task)
    final_challenges.append(task)
    random.shuffle(recover_list)



for _ in range(challenge_bosses):
    boss_challenge  = random.choice(boss_challenges)
    boss            = random.choice(allBosses)
    boss_challenges.remove(boss_challenge)
    allBosses.remove(boss)
    final_challenges.append(f"{boss} → {boss_challenge}")
    random.shuffle(boss_challenges)
    random.shuffle(allBosses)


for _ in range(standard_bosses):
    boss            = random.choice(allBosses)
    allBosses.remove(boss)
    final_challenges.append(f"{boss}")
    random.shuffle(allBosses)


for _ in range(3):  #### Final array shuffle
    random.shuffle(final_challenges)

#### Print / write on file

In [15]:
if lauching_from_gDrive == "no":
    print(f'Starting class: {random.choice(startingClasses)} with {random.choice(startingGifts)} {len(final_challenges)} entries\n {final_challenges}')

    with open(output_file, "w") as f:
        for i in range(0, len(final_challenges), boardSize):
            row = final_challenges[i:i+boardSize]
            f.write("\t".join(row) + "\n")

    print(f"Final challenge board written to: {output_file}")

Starting class: Sorcerer with Master Key 25 entries
 ['Gwyn', 'Talk to Kaathe', 'Have 5 twinkling titanite', 'Find a Soul of a Hero', 'Have 5 demon titanite', 'Obtain Drake Sword', 'Taurus Demon → Cursed', "Forge Smough's Hammer", 'Get a white titanite slab', 'Betray 3 Covenants', 'Kill a black knight (Undead Parish)', 'Obtain Flame Stoneplate Ring', 'Obtain Sanctum', 'Obtain Ring of Steel Protection', 'Four Kings → No Heal', 'Discard 2 Firekeepers souls', 'Quelaag', 'Obtain Stone Armor Set', 'Kill 2 different bosses with no rings', 'Obtain Sunlight Talisman', "Obtain Old Witch's Ring", 'Moonlight Butterfly', 'Get intoxicated', 'Obtain Great Club', 'Dark sign with 30k+ souls']
Final challenge board written to: /home/federico/Nextcloud/Python/scripts/Games/DarkSouls_Minigames/Challenges/final_challenge_board.txt


#

### _____________________________________________________________________________

# Google drive <> sheet setup

In [16]:
if lauching_from_gDrive == "no":
    print(f'My job here is done')
    quit(keep_kernel=True)

My job here is done


In [17]:
import importlib.util
import subprocess
import sys

#### Download packages on new session only
def ensure_package(pkg):
    if importlib.util.find_spec(pkg) is None:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
    else:
        print(f"{pkg} already installed")

ensure_package("gspread")
ensure_package("gspread_formatting")


gspread already installed
gspread_formatting already installed


In [18]:
#### Google Sheets authorization
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
from gspread_formatting import *
from datetime import datetime
import gspread.utils as gutils

#### Connect to Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open("DS Bingo")

ModuleNotFoundError: No module named 'google.colab'

#### Write to the sheet

In [ ]:
#### Generate unique tab name (Run_YYYY-MM-DD_HH-MM)
base_name = "Run_" + datetime.now().strftime("%Y-%m-%d_%H-%M")
existing_titles = [ws.title for ws in spreadsheet.worksheets()]

if base_name not in existing_titles:
    tab_name = base_name
else:
    suffix = 2
    while f"{base_name}_{suffix}" in existing_titles:
        suffix += 1
    tab_name = f"{base_name}_{suffix}"

worksheet = spreadsheet.add_worksheet(title=tab_name, rows=str(boardSize), cols=str(boardSize))



#### Fill the board
rows = [final_challenges[i:i+boardSize] for i in range(0, len(final_challenges), boardSize)]

#### Resize sheet to fit the summary row
worksheet.resize(rows=len(rows) + 1, cols=boardSize)

#### Update the challenge rows
worksheet.update(rows)


#### Append class + gift merged row
summary_text = (
    f"Starting class:   {random.choice(startingClasses)}   with   {random.choice(startingGifts)}"
)

summary_row = len(rows) + 1
worksheet.resize(rows=summary_row, cols=boardSize)

#### Write text in A
worksheet.update_cell(summary_row, 1, summary_text)

#### Merge A:lastColumn on the summary row
last_col_letter = gutils.rowcol_to_a1(1, boardSize).rstrip("1")
merge_range = f"A{summary_row}:{last_col_letter}{summary_row}"

worksheet.merge_cells(merge_range)

### Format

In [ ]:
cell_fmt = cellFormat(
    backgroundColor=color(0.33, 0.33, 0.33),
    textFormat=textFormat(bold=False, foregroundColor=color(1, 1, 1), fontSize=15),
    horizontalAlignment='CENTER',
    verticalAlignment='MIDDLE',
    wrapStrategy='WRAP',
    borders=Borders(
        top=Border('SOLID', color=color(1,1,1)),
        bottom=Border('SOLID', color=color(1,1,1)),
        left=Border('SOLID', color=color(1,1,1)),
        right=Border('SOLID', color=color(1,1,1)),
    )
)

#### Format the whole challenge grid
end_cell = gutils.rowcol_to_a1(len(rows), boardSize)
format_cell_ranges(worksheet, [(f"A1:{end_cell}", cell_fmt)])

#### Format summary row (Cell A_<last>)
format_cell_range(worksheet, merge_range, cell_fmt)

#### Match the format of the other cells
format_cell_range(worksheet, merge_range, cell_fmt)


#### Column widths
for c in range(1, boardSize+1):
    col_letter = gutils.rowcol_to_a1(1, c).rstrip("1")  #### "A", "B", ...
    set_column_width(worksheet, f"{col_letter}:{col_letter}", 220)

#### Row heights (include summary row)
for r in range(1, len(rows)+2):  #### +1 for summary row
    set_row_height(worksheet, f"{r}:{r}", 120)